## Final Assignment


Before working on this assignment please read these instructions fully. Use blackboard to submit a link to your repository. 

On blackboard your find the assessment criteria. Please familiarize yourself with the criteria before beginning the assignment.

This assignment requires that you to find at least two datasets on the web which are related, and that you build an application that visualize these datasets to answer a research question with the broad topic of **health** or **agriculture** in the **region where you were born**. The region can be a city, town or a provence.  

The research question should be a question with a causual nature. For instance questions like: How does independent variable X influence the dependent variable of Y?

The code should be programmed efficiently. Also identify the most critical part and write software test for this part. Take into account the performance of the dataprocessing

### About the data

You can merge these datasets with data from different regions if you like. For instance, you might want to compare the health effect of earhtquacks in Groningen versus Los Angelos USA. 

You are welcome to choose datasets at your discretion, but keep in mind they will be shared with others, so choose appropriate datasets. You are welcome to use datasets of your own as well, but minimual two datasets should be coming from the web and or API's. 

Also, you are welcome to preserve data in its original language, but for the purposes of grading you should provide english translations in your visualization. 

### Instructions:

Define a research question, select data and code your data acquisition, data processing, data analysis and visualization. Write code to test most critical parts. Use a repository with a commit strategy and write a readme file. 

Write a small document with the following:
- State the region and the domain category that your data sets are about 
- State the research question 
- Justify the chosen data storage and processing approach
- Justify the chosen analysis approach
- Justify the chosen data visualization approach

Upload your document and the link of your repository to black board

In [23]:
# imports 
import numpy as np
import pandas as pd
import os
import re
import math

import pandas as pd
import numpy as np
from bokeh.palettes import magma
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models import HoverTool,WheelZoomTool, PanTool, ResetTool
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

# Introduction

In this study the relationship of influenza cases and influenza variants are assessed by considering the population density for the province Baden-Württemberg (Germany). Influenza cases are recorded by the Robert - Koch institute beginning with season 2000/2001. A season lasts from september of the first year till end of august of the next year. The case numbers were collected per season and virus variant/type for each district of Baden-Württemberg. There are in total 44 districts, which differ in population density. Therefore population size, area size and population density were obtained for each district from a second data source (statistiches Landesamt Baden-Württemberg). The recordings are available starting in 1961 and are available until 2020.

The available joined data provides seasonal information on the case numbers per influenza virus type for each district as well as population density for each district. This study aims to analyse general seasonal trends for whole Baden-Württemberg, detecting differences in the spreading of influenza between densly populated areas and sparsly popualted areas in Baden-Württemberg as well as analyzing trends in virus variation over time and spreading on district level.

This sums up to the following hypotheses:
- the prevalence of influenza cases genereally increased, with a strong increase from the year 2012 onwards (sudden increase in virus variant diversity, introducing new clades/strains) and with sudden decrease in 2020 due to the covid measurments
- the spreading (reflected in case numbers) is significantly different comparing sparsly and densly popualted areas
- the virus diversity increase leads to higher case numbers and more different virus variants recorded per district

# Load, inspect and process the population density data

source: statistisches Landesamt Baden-Württemberg

In [24]:
# create dictionary with abbreviation for all districts
districts_dict = {'LK Alb Donau': 'AD', 'SK Baden - Baden': 'BAD', 'LK Biberach': 'BC', 'LK Boeblingen': 'BB', 
                  'LK Bodensee': 'BS', 'LK Breisgau - Hochschwarzwald': 'BHS', 'LK Calw': 'CW', 'LK Emmendingen': 'EM',
                 'LK Enzkreis': 'ENZ', 'LK Esslingen': 'ES', 'SK Freiburg': 'FR', 'LK Freudenstadt': 'FDS', 'LK Goeppingen': 'GP',
                 'SK Heidelberg': 'HD', 'LK Heidenheim': 'HDH', 'SK Heilbronn': 'HNS', 'LK Heilbronn': 'HNL', 'LK Hohenlohe': 'KUEN',
                 'SK Karlsruhe': 'KAS', 'LK Karlsruhe': 'KAL', 'LK Konstanz': 'KN', 'LK Loerrach': 'LOE', 'LK Ludwigsburg': 'LW', 
                  'LK Main - Tauber': 'MT', 'SK Mannheim': 'MA', 'LK Neckar - Odenwald': 'NO', 'LK Ortenau': 'OG', 
                  'LK Ostalbkreis': 'OA', 'SK Pforzheim': 'PF', 'LK Rastatt': 'RA', 'LK Ravensburg': 'RV', 'LK Rems - Murr': 'RM',
                 'LK Reutlingen': 'RT', 'LK Rhein - Neckar': 'RN', 'LK Rottweil': 'RW', 'LK Schwaebisch - Hall': 'SHA',
                 'LK Schwarzwald - Baar': 'VS', 'LK Sigmaringen': 'SIG', 'SK Stuttgart': 'S', 'LK Tuebingen': 'TUE',
                 'LK Tuttlingen': 'TUT', 'SK Ulm': 'U', 'LK Waldshut': 'WT', 'LK Zollernalb': 'BL'}

In [25]:
# import the population density datasets

# define header list
col_name_area = ['year', 'area [ha]', 'population', 'population_density', 'population_density_bw']

# sort the dictionary to match the alphabetic order of the files in the folder to be read
sorted_districts = {key: value for key, value in sorted(districts_dict.items())}

# store file path
path = 'C:/Data_Science_for_Life_Sciences_MASTER/programming1/area_population_data'

# regex matching all the dataset files
regex = re.compile('[LS]K_(\S)+\.csv')

d = {}
l = []

# loop over everything found in the given path, then loop over all files  if they match the regex append them to a list
for root, dirs, files in os.walk(path):
    for item in files:
        if regex.match(item):
            l.append(item)

# loop over the file list and the district-name dictionary, read the file and add the current district name as key and the dataframe as value to dictionary
for file, name in zip(l, sorted_districts.keys()):
    df = pd.read_csv(path + '/' + file, sep = ';', thousands = '.', header = None, skiprows = 4, nrows = 63, names = col_name_area)
    df['year'] = df['year'].str.slice_replace(4,6,'')
    d[f'{name}'.format(name)] = df

d['LK Emmendingen']

,year,area [ha],population,population_density,population_density_bw
0,1961,68106,104372,153,217
1,1961,68106,105285,155,219
2,1962,68106,106694,157,224
3,1963,68107,108235,159,227
4,1964,68107,110048,162,231
...,...,...,...,...,...
58,2016,67979,163251,240,307
59,2017,67979,164712,242,309
60,2018,67980,165383,243,310
61,2019,67980,166408,245,311


In [26]:
# produce the district column
for key in d:
    d[key]['district'] = districts_dict[key]

d['LK Alb Donau']

,year,area [ha],population,population_density,population_density_bw,district
0,1961,136806,126674,93,217,AD
1,1961,136806,128002,94,219,AD
2,1962,136806,131235,96,224,AD
3,1963,136806,133560,98,227,AD
4,1964,136806,136384,100,231,AD
...,...,...,...,...,...,...
58,2016,135855,193318,142,307,AD
59,2017,135855,194629,143,309,AD
60,2018,135855,196047,144,310,AD
61,2019,135854,197076,145,311,AD


In [27]:
# check data types
d['LK Alb Donau'].dtypes

year                     object
area [ha]                 int64
population                int64
population_density        int64
population_density_bw     int64
district                 object
dtype: object

In [28]:
# typecast year
for key in d:
    d[key]['year'] = d[key]['year'].map(int)

# check whether it worked
d['SK Ulm'].dtypes

year                      int64
area [ha]                 int64
population                int64
population_density        int64
population_density_bw     int64
district                 object
dtype: object

In [29]:
# filter for the relevant years
relevant_years = [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

for key in d:
    d[key] = d[key][d[key].year.isin(relevant_years)]


d['SK Karlsruhe']

,year,area [ha],population,population_density,population_density_bw,district
42,2000,17346,278558,1606,294,KAS
43,2001,17346,279578,1612,297,KAS
44,2002,17346,281334,1622,298,KAS
45,2003,17346,282595,1629,299,KAS
46,2004,17346,284163,1638,300,KAS
47,2005,17346,285263,1645,300,KAS
48,2006,17346,286327,1651,300,KAS
49,2007,17346,288917,1666,301,KAS
50,2008,17346,290736,1676,301,KAS
51,2009,17346,291959,1683,301,KAS


The information for the population density is available up to the year 2020, however the influenza dataset provides information for year/season of 2021 as well. For that reason the popualtion density for this year needs to be extrapolated. The popualtion size developement is assumed to increase linearly. The next code block is viusally checking the trends for population growth to see whether extrapolation is easily possible for the year 2021, and which method is fitting best.

In [30]:
# plot the population size over the years to see the general trend
line_color = magma(44)

p1 = figure(title='Population size per district',  x_axis_label = 'year', y_axis_label = 'population')

for name, color in zip(d.keys(), line_color):
    p1.line(x = d[name].year, y = d[name].population, legend_label= districts_dict[name], color = color)

show(p1)

the general growth seems to be linear with some fluctuations after 2015. For the year 2011 an odd pattern is found in all of the curves. Checking the meta data of the files, it was explained that the method of measuring/estimating the population size was updated in 2011. The population data is an extrapolation which was from 2011 onwards based on the "mikrozensus". Concluding from this, the population size measured from 2011 onwards seems to be more accurate and the values before should be discarded an extrapolated as well. Considering the fluctuations a curve fit was considered the better option instead of extrapolating through all the points exactly. A second plot, showing area size over population should reveal whether there is a linear dependency between the three variables (population size, area and the resulting population density).

In [31]:
p2 = figure(title='population density',  x_axis_label = 'area [ha]', y_axis_label = 'population')

for name, color in zip(d.keys(), line_color):
    source = ColumnDataSource(d[name])
    p2.line(x = 'area [ha]', y = 'population', color = color, source = source)
    p2.add_tools(HoverTool(show_arrow=False, line_policy='next', tooltips=[
    ('district', '@district')]))

show(p2)

There is a constant growth in population for constant area size observed for all districts except LK Reutlingen. It seems that the linear apporach for curve fitting is fine. 

In [32]:
d['LK Reutlingen']

,year,area [ha],population,population_density,population_density_bw,district
42,2000,109413,277995,254,294,RT
43,2001,109414,279177,255,297,RT
44,2002,109414,280613,256,298,RT
45,2003,109414,281690,257,299,RT
46,2004,109414,281779,258,300,RT
47,2005,109414,282049,258,300,RT
48,2006,109404,281891,258,300,RT
49,2007,109405,281580,257,301,RT
50,2008,109404,281080,257,301,RT
51,2009,109404,280927,257,301,RT


Referring back to the metadata in the datafile for Reutlingen, there is stated, that an area was not belonging to this district anymore from 2011 onwards, while the population growth seems to be linear. The loss of area for this district led to small area increases for the neighboring districts, which causes fluctuations as well. These fluctuations, however seem to be too small to catch in the visualisation. For further simplification and balancing this fluctuations out, the average of the area was formed and used as "extrapolated" value on which the further calucaltions were based.

For proceeding further, decisison was made to conduct linear curve fitting based on the year 2011 - 2020 and extrapolate for the range 2000 - 2010 and for the year 2021. 

Furthermore, the LK Reutlingen will be excluded from the further analysis.

In [33]:
for key in d:
    d[key]['pop_exp'] = np.zeros(len(d[key]))
    d[key]['pop_dens_exp'] = np.zeros(len(d[key]))
    d[key]['area_exp'] = d[key]['area [ha]'].mean().round(2)
    d[key]['area [km^2]'] = d[key]['area_exp']/100
    d[key]['area ex [km^2]'] = np.repeat(d[key]['area [km^2]'].mean().round(2), len(d[key]))

d['LK Alb Donau']

C:\Users\chiar\AppData\Local\Temp/ipykernel_20848/2433236665.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[key]['pop_exp'] = np.zeros(len(d[key]))
C:\Users\chiar\AppData\Local\Temp/ipykernel_20848/2433236665.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[key]['pop_dens_exp'] = np.zeros(len(d[key]))
C:\Users\chiar\AppData\Local\Temp/ipykernel_20848/2433236665.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

,year,area [ha],population,population_density,population_density_bw,district,pop_exp,pop_dens_exp,area_exp,area [km^2],area ex [km^2]
42,2000,135732,185929,137,294,AD,0.0,0.0,135793.33,1357.9333,1357.93
43,2001,135732,187000,138,297,AD,0.0,0.0,135793.33,1357.9333,1357.93
44,2002,135732,188146,139,298,AD,0.0,0.0,135793.33,1357.9333,1357.93
45,2003,135732,189101,139,299,AD,0.0,0.0,135793.33,1357.9333,1357.93
46,2004,135731,189717,140,300,AD,0.0,0.0,135793.33,1357.9333,1357.93
47,2005,135733,190233,140,300,AD,0.0,0.0,135793.33,1357.9333,1357.93
48,2006,135732,190189,140,300,AD,0.0,0.0,135793.33,1357.9333,1357.93
49,2007,135732,190212,140,301,AD,0.0,0.0,135793.33,1357.9333,1357.93
50,2008,135732,190403,140,301,AD,0.0,0.0,135793.33,1357.9333,1357.93
51,2009,135732,189884,140,301,AD,0.0,0.0,135793.33,1357.9333,1357.93


In [34]:
#create row for the year 2021 respectively season 2021/22
dict_21 = {'year': [2021], 'area [ha]': np.zeros(1), 'population': np.zeros(1), 'population_density': np.zeros(1), 
         'population_density_bw': np.zeros(1), 'district': np.zeros((1), dtype = '|S4'), 'pop_exp': np.zeros(1), 
         'pop_dens_exp': np.zeros(1), 'area [km^2]': np.zeros(1), 'area ex [km^2]': np.zeros(1)} 

df_21 = pd.DataFrame(dict_21)
df_21

,year,area [ha],population,population_density,population_density_bw,district,pop_exp,pop_dens_exp,area [km^2],area ex [km^2]
0,2021,0.0,0.0,0.0,0.0,b'',0.0,0.0,0.0,0.0


In [35]:
# append this dataframe to each dataframe and fill the values except the ones which will be interpolated
for key in d.keys():
    df_21.district = sorted_districts[key]
    df_21['area [ha]'] = np.nan
    df_21.population = np.nan
    df_21.population_density = np.nan
    df_21.population_density_bw = np.nan
    df_21['area [km^2]'] = np.nan
    df_21['area ex [km^2]'] = np.repeat(d[key]['area ex [km^2]'].unique(), 1)
    d[key] = d[key].append(df_21, ignore_index=True)
    
d['SK Mannheim']

,year,area [ha],population,population_density,population_density_bw,district,pop_exp,pop_dens_exp,area_exp,area [km^2],area ex [km^2]
0,2000,14496.0,306729.0,2116.0,294.0,MA,0.0,0.0,14496.24,144.9624,144.96
1,2001,14496.0,308385.0,2127.0,297.0,MA,0.0,0.0,14496.24,144.9624,144.96
2,2002,14496.0,308759.0,2130.0,298.0,MA,0.0,0.0,14496.24,144.9624,144.96
3,2003,14496.0,308353.0,2127.0,299.0,MA,0.0,0.0,14496.24,144.9624,144.96
4,2004,14496.0,307499.0,2121.0,300.0,MA,0.0,0.0,14496.24,144.9624,144.96
5,2005,14496.0,307900.0,2124.0,300.0,MA,0.0,0.0,14496.24,144.9624,144.96
6,2006,14496.0,307914.0,2124.0,300.0,MA,0.0,0.0,14496.24,144.9624,144.96
7,2007,14496.0,309795.0,2137.0,301.0,MA,0.0,0.0,14496.24,144.9624,144.96
8,2008,14496.0,311342.0,2148.0,301.0,MA,0.0,0.0,14496.24,144.9624,144.96
9,2009,14496.0,311969.0,2152.0,301.0,MA,0.0,0.0,14496.24,144.9624,144.96


In [37]:
d['SK Mannheim'].isnull().sum()

year                     0
area [ha]                1
population               1
population_density       1
population_density_bw    1
district                 0
pop_exp                  0
pop_dens_exp             0
area_exp                 1
area [km^2]              1
area ex [km^2]           0
dtype: int64

The linear curve fit is conducted choosing the years 2011 - 2020 as range, extrapolating range is 2000 to 2022 in one year steps. Then the according values are written to list and provided together with the base years to the polyfit function with degree 1. The resulting coefficients are used to conduct the fit with polynom of first order. Than the populaiton size values are extrapolated using the curvefit object. Lastly, the popualiton density is re-calculated based on the extrapolated population size vlaues.

In [41]:
# Linear curve fit 
basis_years = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
exp_years = np.linspace(2000, 2021, 22)

In [42]:
### eleganter mit true/false variable ###
for name in d.keys():
    population = d[name][d[name].year.isin(basis_years)].population.tolist()
    coef = np.polyfit(basis_years, population, deg = 1)
    curvefit = np.poly1d(coef)
    y = curvefit(exp_years).round(2)
    d[name]['pop_exp'] = y
    d[name]['pop_dens_exp'] = (d[name]['pop_exp']/d[name]['area ex [km^2]']).round(2)
d['LK Alb Donau'] 

,year,area [ha],population,population_density,population_density_bw,district,pop_exp,pop_dens_exp,area_exp,area [km^2],area ex [km^2]
0,2000,135732.0,185929.0,137.0,294.0,AD,170214.78,125.35,135793.33,1357.9333,1357.93
1,2001,135732.0,187000.0,138.0,297.0,AD,171632.96,126.39,135793.33,1357.9333,1357.93
2,2002,135732.0,188146.0,139.0,298.0,AD,173051.15,127.44,135793.33,1357.9333,1357.93
3,2003,135732.0,189101.0,139.0,299.0,AD,174469.33,128.48,135793.33,1357.9333,1357.93
4,2004,135731.0,189717.0,140.0,300.0,AD,175887.51,129.53,135793.33,1357.9333,1357.93
5,2005,135733.0,190233.0,140.0,300.0,AD,177305.69,130.57,135793.33,1357.9333,1357.93
6,2006,135732.0,190189.0,140.0,300.0,AD,178723.87,131.61,135793.33,1357.9333,1357.93
7,2007,135732.0,190212.0,140.0,301.0,AD,180142.05,132.66,135793.33,1357.9333,1357.93
8,2008,135732.0,190403.0,140.0,301.0,AD,181560.24,133.70,135793.33,1357.9333,1357.93
9,2009,135732.0,189884.0,140.0,301.0,AD,182978.42,134.75,135793.33,1357.9333,1357.93


Re-do the visualisations for population growth per year and the dependencies between area and population size, to check whether extrapolation went as expected. Concatenate the dataframes per district into one big dataframe. To merge later to the influenza dataset.

In [43]:
# redo the two plottings to check on the changes
p3 = figure(title='extrpolated population per district',  x_axis_label = 'year', y_axis_label = 'population')

for name, color in zip(d.keys(), line_color):
    p3.line(x = d[name].year, y = d[name].pop_exp, legend_label= districts_dict[name], color = color)

show(p3)

In [45]:
# concatenate the population density data
area_df = pd.concat(d.values())
area_df = area_df[area_df.district != 'RT']

In [47]:
area_df.dtypes

year                       int64
area [ha]                float64
population               float64
population_density       float64
population_density_bw    float64
district                  object
pop_exp                  float64
pop_dens_exp             float64
area_exp                 float64
area [km^2]              float64
area ex [km^2]           float64
dtype: object

In [48]:
p4 = figure(title='population density',  x_axis_label = 'area [ha]', y_axis_label = 'population')

for (name, group), color in zip(area_df.groupby('district'), line_color):
    p4.line(x = group['area ex [km^2]'], y = group['pop_exp'], line_color = color)

show(p4)